<a href="https://colab.research.google.com/github/woongjoonchoi/DeepLearningPaper-Reproducing/blob/master/Vgg/vgg_model_define.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import albumentations as A
import numpy as np

import matplotlib.pyplot as plt
from torchvision.datasets import Caltech256 ,Caltech101 ,CIFAR100
import os
from PIL import Image
from urllib.request import urlretrieve
import requests
import tarfile

In [ ]:
from torch import nn

In [ ]:
import torch

In [ ]:
# With square kernels and equal stride
c = nn.Conv2d(16, 33, 3, stride=2)
# non-square kernels and unequal stride and with padding
m = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2))
d = nn.Conv2d(3,44,4)
a=[]
kk =[c,m]
dd = [d]
a += kk
a += dd
# li = nn.Sequential(*a)

li = nn.ModuleList(a)

# non-square kernels and unequal stride and with padding and dilation
# m = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2), dilation=(3, 1))
input = torch.ones(20, 16, 50, 100)
output = m(input)



In [ ]:
a

[Conv2d(16, 33, kernel_size=(3, 3), stride=(2, 2)),
 Conv2d(16, 33, kernel_size=(3, 5), stride=(2, 1), padding=(4, 2)),
 Conv2d(3, 44, kernel_size=(4, 4), stride=(1, 1))]

In [ ]:
li

ModuleList(
  (0): Conv2d(16, 33, kernel_size=(3, 3), stride=(2, 2))
  (1): Conv2d(16, 33, kernel_size=(3, 5), stride=(2, 1), padding=(4, 2))
  (2): Conv2d(3, 44, kernel_size=(4, 4), stride=(1, 1))
)

In [ ]:
output.shape

torch.Size([20, 33, 28, 100])

In [ ]:
Config_channels = {
"A" : [64,"M" , 128,  "M"  , 256,256,"M" ,512,512 ,"M" , 512,512,"M"] ,
"A_lrn" : [64,"LRN","M" , 128,  "M"  , 256,256,"M" ,512,512 ,"M" , 512,512,"M"] ,
"B" :[64,64,"M" , 128,128,  "M"  , 256,256,"M" ,512,512 ,"M" , 512,512,"M"]  ,
"C" : [64,64,"M" , 128,128,  "M"  , 256,256,256,"M" ,512,512 ,512,"M" , 512,512,512,"M"] ,
"D" :[64,64,"M" , 128,128,  "M"  , 256,256,256,"M" ,512,512 ,512,"M" , 512,512,512,"M"] ,
"E" :[64,64,"M" , 128,128,  "M"  , 256,256,256,256,"M" ,512,512 ,512,512,"M" , 512,512,512,512,"M"]         ,

}

In [ ]:
Config_kernel = {
"A" : [3,2 , 3,  2  , 3,3,2 ,3,3 ,2 , 3,3,2] ,
"A_lrn" : [3,2,2 , 3,  2  , 3,3,2 ,3,3 ,2 , 3,3,2] ,
"B" :[3,3,2 , 3,3,  2  , 3,3,2 ,3,3 ,2 , 3,3,2]  ,
"C" : [3,3,2 , 3,3,  2  , 3,3,1,2 ,3,3 ,1,2 , 3,3,1,2] ,
"D" :[3,3,2 , 3,3,  2  , 3,3,3,2 ,3,3 ,3,2 , 3,3,3,2] ,
"E" :[3,3,2 , 3,3,  2  , 3,3,3,3,2 ,3,3 ,3,3,2 , 3,3,3,3,2]         ,

}

In [ ]:
def make_feature_extractor(cfg_c,cfg_k):
    feature_extract = []
    in_channels = 3
    for out_channels , kernel in zip(cfg_c,cfg_k) :
        if out_channels == "M" :
            feature_extract += [nn.MaxPool2d(kernel,2) ]
        else :
            feature_extract+= [nn.Conv2d(in_channels,out_channels,kernel,stride = 1 , padding = 1) , nn.Relu()]
        in_channels = out_channels
    return nn.ModuleList(feature_extract)


In [ ]:
class Model_vgg(nn.module) :
    def __init__(self,version , num_classes):
        conv_5_out_w ,conv_5_out_h = 7,7
        conv_5_out_dim =512
        conv_1_by_1_1_outchannel = 4096
        conv_1_by_1_2_outchannel = 4096
        # conv_1_by_1_3_outchannel = num_classes
        self.feature_extractor = make_feature_extractor(Config_channels[version] , Config_kernel[version])

        self.output_layer = nn.Sequential(
                             nn.Conv2d(conv_5_out_dim * conv_5_out_w *conv_5_out_h ,conv_1_by_1_1_outchannel ,1,padding = 1) ,
                             nn.Relu(),
                             nn.Dropout(),
                             nn.Conv2d(conv_1_by_1_1_outchannel ,conv_1_by_1_2_outchannel,1,padding =1 ) ,
                             nn.Relu(),
                             nn.Dropout(),
                             nn.Conv2d(conv_1_by_1_2_outchannel ,num_classes,1,padding =1 )
                             )

    def forward(self,x):
        x = self.feature_extractor(x)
        x = self.output_layer(x)


    #     pass

